<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/reinlearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install tf-agents[reverb]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [177]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import pandas as pd
import reverb

from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tf_agents.drivers import py_driver
from tf_agents.utils import common
import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import sequential
from tf_agents.specs import tensor_spec
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.policies import random_tf_policy

In [167]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

In [108]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv')
df.pop("index")
del df[df. columns[0]]
df = df.astype('float64')
min = df.min()
obsShape = len(df. columns)

dfarray = df.to_numpy()
dflist = dfarray.tolist()

length = len(dflist)

print(dflist[1][3])
print(df)
print(np.asarray(dflist).shape)
#print(df.dtypes)

-0.0001999999999999
       Direction  Bottom_Point  Top_Point  Relative_Change     Size   day  \
0            0.0      0.000000   0.666667         -0.00080  0.00120   2.0   
1            0.0      0.444444   0.666667         -0.00020  0.00090   2.0   
2            1.0      0.230769   0.846154          0.00080  0.00130   2.0   
3            0.0      0.888889   0.888889          0.00000  0.00090   2.0   
4            1.0      0.400000   0.700000          0.00030  0.00100   2.0   
...          ...           ...        ...              ...      ...   ...   
93075        1.0      0.575000   0.962500          0.00093  0.00240  29.0   
93076        1.0      0.457317   0.682927          0.00037  0.00164  29.0   
93077        0.0      0.422680   0.824742         -0.00039  0.00097  29.0   
93078        1.0      0.634146   0.780488          0.00006  0.00041  29.0   
93079        1.0      0.515152   0.969697          0.00015  0.00033  29.0   

       month  weekday  hour_of_day  
0        5.0      

In [152]:
class TradingEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,9), dtype=np.float32, name='observation')
    self.counter = 0
    self.reward = 0
    self._state = dflist[0]
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = dflist[0]
    self._episode_ended = False
    self.reward = 0
    self.counter = 0
    return ts.restart(np.array([self._state], dtype=np.float32))

  def _step(self, action):

    self.counter += 1

    self._state = dflist[self.counter]

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Buy Sell reward.
    if action == 1:
      self.reward += dflist[self.counter][3] 
    elif action == 0:
      self.reward -= dflist[self.counter][3]
    else:
      raise ValueError('`action` should be 0 or 1.')


    if self.counter == 1000:
      self._episode_ended = True


    if self._episode_ended:
      return ts.termination(np.array([self._state], dtype=np.float32), self.reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.float32), reward=self.reward, discount=0)

In [153]:
environment = TradingEnv()
utils.validate_py_environment(environment, episodes=2)

In [155]:
Buy = np.array(0, dtype=np.int32)
end_round_action = np.array(1, dtype=np.int32)

environment = TradingEnv()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(12):
  time_step = environment.step(Buy)
  print(time_step)
  cumulative_reward += time_step.reward

time_step = environment.step(end_round_action)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([[ 0.000000e+00,  0.000000e+00,  6.666667e-01, -8.000000e-04,
         1.200000e-03,  2.000000e+00,  5.000000e+00,  0.000000e+00,
         0.000000e+00]], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})
TimeStep(
{'discount': array(0., dtype=float32),
 'observation': array([[ 0.0000000e+00,  4.4444445e-01,  6.6666669e-01, -1.9999999e-04,
         8.9999998e-04,  2.0000000e+00,  5.0000000e+00,  0.0000000e+00,
         1.0000000e+00]], dtype=float32),
 'reward': array(0.0002, dtype=float32),
 'step_type': array(1, dtype=int32)})
TimeStep(
{'discount': array(0., dtype=float32),
 'observation': array([[1.0000000e+00, 2.3076923e-01, 8.4615386e-01, 7.9999998e-04,
        1.3000000e-03, 2.0000000e+00, 5.0000000e+00, 0.0000000e+00,
        2.0000000e+00]], dtype=float32),
 'reward': array(-0.0006, dtype=float32),
 'step_type': array(1, dtype=int32)})
TimeStep(
{'discount': array(0.

In [156]:
env = tf_py_environment.TFPyEnvironment(environment)
print(isinstance(env, tf_environment.TFEnvironment))
print("TimeStep Specs:", env.time_step_spec())
print("Action Specs:", env.action_spec())

True
TimeStep Specs: TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(1, 9), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32)),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})
Action Specs: BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(1, dtype=int32))


In [157]:
print('Action Spec:')
print(env.action_spec())

Action Spec:
BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(1, dtype=int32))


In [158]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [Flatten(), q_values_layer])

In [159]:
learning_rate = .001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [160]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

In [161]:
random_policy = random_tf_policy.RandomTFPolicy(env.time_step_spec(),
                                                env.action_spec())

In [162]:
compute_avg_return(env, random_policy, 10)

-4.2019286

In [171]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

In [174]:
agent.collect_data_spec

Trajectory(
{'action': BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(1, dtype=int32)),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': BoundedTensorSpec(shape=(1, 9), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32)),
 'policy_info': (),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [173]:
agent.collect_data_spec._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

In [178]:
#@test {"skip": true}
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(env.reset())

(TimeStep(
 {'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
  'observation': <tf.Tensor: shape=(1, 1, 9), dtype=float32, numpy=
 array([[[ 0.0000000e+00,  4.4444445e-01,  8.8888890e-01, -3.9999999e-04,
           8.9999998e-04,  6.0000000e+00,  5.0000000e+00,  4.0000000e+00,
           4.0000000e+00]]], dtype=float32)>,
  'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.00856], dtype=float32)>,
  'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>}),
 ())

In [179]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

<_PrefetchDataset element_spec=(Trajectory(
{'action': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None),
 'discount': TensorSpec(shape=(64, 2), dtype=tf.float32, name=None),
 'next_step_type': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None),
 'observation': TensorSpec(shape=(64, 2, 1, 9), dtype=tf.float32, name=None),
 'policy_info': (),
 'reward': TensorSpec(shape=(64, 2), dtype=tf.float32, name=None),
 'step_type': TensorSpec(shape=(64, 2), dtype=tf.int32, name=None)}), SampleInfo(key=TensorSpec(shape=(64, 2), dtype=tf.uint64, name=None), probability=TensorSpec(shape=(64, 2), dtype=tf.float64, name=None), table_size=TensorSpec(shape=(64, 2), dtype=tf.int64, name=None), priority=TensorSpec(shape=(64, 2), dtype=tf.float64, name=None), times_sampled=TensorSpec(shape=(64, 2), dtype=tf.int32, name=None)))>

In [180]:
iterator = iter(dataset)
print(iterator)

In [ ]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)